<a href="https://colab.research.google.com/github/jdasam/mas1004-2022/blob/main/notebooks/Data_AI_3rd_week2_spiral_livecoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
from math import pi

In [ ]:
# if you run this code in your local computer, you have to check that torch is installed
!pip install torch
# But in Google Colab, torch is already installed 

In [2]:
# make spiral data
# https://gist.github.com/45deg/e731d9e7f478de134def5668324c44c5
N = 500
theta = np.sqrt(np.random.rand(N))*2*pi # np.linspace(0,2*pi,100)

r_a = 2*theta + pi
data_a = np.array([np.cos(theta)*r_a, np.sin(theta)*r_a]).T
x_a = data_a + np.random.randn(N,2)

r_b = -2*theta - pi
data_b = np.array([np.cos(theta)*r_b, np.sin(theta)*r_b]).T
x_b = data_b + np.random.randn(N,2)

res_a = np.append(x_a, np.zeros((N,1)), axis=1)
res_b = np.append(x_b, np.ones((N,1)), axis=1)

res = np.append(res_a, res_b, axis=0)
np.random.shuffle(res)

In [3]:
import matplotlib.pyplot as plt
def visualize_decision_boundary(model, data, label):
  x_linspace = torch.linspace(min(data[:,0]), max(data[:,0]), steps=200)
  y_linspace = torch.linspace(min(data[:,1]), max(data[:,1]), steps=200)
  grid_x, grid_y = torch.meshgrid(x_linspace, y_linspace)
  grid_xy = torch.stack([grid_x, grid_y]).permute(1,2,0)
  grid_xy = grid_xy.view(-1, 2)
  if isinstance(model, torch.nn.Module):
    value_by_grid = model(grid_xy)
  else:
    value_by_grid = run_neuron(model, grid_xy)
  value_by_grid = value_by_grid.view(200, 200, 1)
  value_by_grid[value_by_grid<=0.5] = 0
  value_by_grid[value_by_grid>0.5] = 1

  plt.scatter(x=data[label[:,0]==0,0], y=data[label[:,0]==0,1])
  plt.scatter(x=data[label[:,0]==1,0], y=data[label[:,0]==1,1])

  plt.contourf(grid_x.detach().numpy(), grid_y.detach().numpy(), value_by_grid.detach().numpy().squeeze(), alpha=0.3)

In [33]:
# let's convert our data in tensor
datas = torch.tensor(res, dtype=torch.float32)
# represent each number with 32 bits, instead of 64 bits

In [34]:
datas # a tensor can only have one data type.
# so if you want to represent float data
# you have to represent even integer in float

tensor([[ -9.7528,   1.3513,   0.0000],
        [ 13.8056,  -4.6267,   0.0000],
        [ 11.5181,  -9.6006,   0.0000],
        ...,
        [-15.1413,   0.9364,   1.0000],
        [  8.6965, -10.3705,   0.0000],
        [ 17.1500,  -0.6040,   0.0000]])

In [35]:
datas[datas[:,-1]==0]

tensor([[ -9.7528,   1.3513,   0.0000],
        [ 13.8056,  -4.6267,   0.0000],
        [ 11.5181,  -9.6006,   0.0000],
        ...,
        [ -7.4159, -11.0369,   0.0000],
        [  8.6965, -10.3705,   0.0000],
        [ 17.1500,  -0.6040,   0.0000]])

In [36]:
data_xy = datas[:, :2]
data_label= datas[:, -1]

In [37]:
data_xy, data_label[:10]

(tensor([[ -9.7528,   1.3513],
         [ 13.8056,  -4.6267],
         [ 11.5181,  -9.6006],
         ...,
         [-15.1413,   0.9364],
         [  8.6965, -10.3705],
         [ 17.1500,  -0.6040]]),
 tensor([0., 0., 0., 1., 0., 0., 0., 1., 1., 0.]))

In [38]:
# How to make a tensor in specific shape
# Let's make 2 x 3 tensor
print(torch.zeros(2,3))
print(torch.ones(2,3))
print(torch.rand(2,3)) # every element will have random value between 0-1
print(torch.randn(2,3)) # this uses gaussian (normal) distribution
 

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0.0152, 0.9159, 0.7349],
        [0.6722, 0.1061, 0.5764]])
tensor([[ 0.3416, -0.9599, -2.6229],
        [-0.3976,  0.0559,  1.7811]])


In [42]:
# Let's make a neural network layer


class Layer:
  def __init__(self, in_features, num_neurons):
    # in_features: how many features does this layer take as an input
    # 
    self.in_features = in_features
    self.num_neurons = num_neurons

    # next step: define a weight matrix
    self.weight = torch.randn(self.in_features, self.num_neurons)
  
  def __call__(self, input):
    return torch.mm(input, self.weight)

layer = Layer(2, 4)
layer.weight

tensor([[ 0.3099, -0.1747,  0.5885, -0.1636],
        [ 0.6864,  1.9186, -0.8930, -0.1188]])

In [18]:
data_xy.shape, layer.weight.shape

(torch.Size([1000, 2]), torch.Size([2, 4]))

In [27]:
# what will be the output's size
output = torch.zeros(data_xy.shape[0], layer.weight.shape[1])

for data_idx, data_sample in enumerate(data_xy):
  for neuron_idx in range(layer.weight.shape[1]):
    temporary_sum = 0
    for feature_idx in range(len(data_sample)):
      data_pos = data_sample[feature_idx]
      corresp_neuron_weight = layer.weight[feature_idx, neuron_idx]
      temporary_sum += data_pos * corresp_neuron_weight
    # print(data_sample[i])
    output[data_idx, neuron_idx] = temporary_sum 

In [28]:
output

tensor([[  0.3134,  -3.3787,   6.2090,   9.5663],
        [  1.0399,   0.5123,  -9.5956, -19.9656],
        [  4.0055,  -8.6053,  -9.7113, -30.2453],
        ...,
        [  1.1214,  -7.0731,   9.2944,  12.1026],
        [  4.7307, -11.4095,  -8.2598, -30.2254],
        [ -1.5197,   8.7299, -10.3918, -12.6274]])

In [39]:
# Easier, and more efficient way
torch.mm(data_xy, layer.weight) # mm means matrix multiplication

tensor([[  0.3134,  -3.3787,   6.2090,   9.5663],
        [  1.0399,   0.5123,  -9.5956, -19.9656],
        [  4.0055,  -8.6053,  -9.7113, -30.2453],
        ...,
        [  1.1214,  -7.0731,   9.2944,  12.1026],
        [  4.7307, -11.4095,  -8.2598, -30.2254],
        [ -1.5197,   8.7299, -10.3918, -12.6274]])

In [40]:
# computation time with for loop vs matrix multiplication
import time


start_time = time.time()

output = torch.zeros(data_xy.shape[0], layer.weight.shape[1])
for data_idx, data_sample in enumerate(data_xy):
  for neuron_idx in range(layer.weight.shape[1]):
    temporary_sum = 0
    for feature_idx in range(len(data_sample)):
      data_pos = data_sample[feature_idx]
      corresp_neuron_weight = layer.weight[feature_idx, neuron_idx]
      temporary_sum += data_pos * corresp_neuron_weight
    # print(data_sample[i])
    output[data_idx, neuron_idx] = temporary_sum

end_time = time.time()
print("time spent: ", end_time - start_time) 

time spent:  0.48915719985961914


In [41]:
# Never tries to use for loop in matrix multiplication
# matrix multiplication is much much faster than for loop

start_time = time.time()
torch.mm(data_xy, layer.weight)
end_time = time.time()
print(end_time - start_time)

0.009114742279052734


In [60]:
def relu(x):
  # x is an tensor
  is_larger_than_zero = x > 0
  new_x = torch.clone(x)
  new_x[~is_larger_than_zero] = 0
  # print(is_larger_than_zero.shape, x.shape)
  # the index of tensor can have the same shape with the tensor itself
  return new_x
relu(output)

tensor([[ 0.3134,  0.0000,  6.2090,  9.5663],
        [ 1.0399,  0.5123,  0.0000,  0.0000],
        [ 4.0055,  0.0000,  0.0000,  0.0000],
        ...,
        [ 1.1214,  0.0000,  9.2944, 12.1026],
        [ 4.7307,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  8.7299,  0.0000,  0.0000]])

In [56]:
output > 0


tensor([[ True, False,  True,  True],
        [ True,  True, False, False],
        [ True, False, False, False],
        ...,
        [ True, False,  True,  True],
        [ True, False, False, False],
        [False,  True, False, False]])

In [61]:
layer_0 = Layer(in_features=2, num_neurons=4)
layer_1 = Layer(in_features=4, num_neurons=1)

print(data_xy.shape)
out_0 = layer_0(data_xy)
print(out_0.shape)
out_1 = layer_1(relu(out_0))
print(out_1.shape)

torch.Size([1000, 2])
torch.Size([1000, 4])
torch.Size([1000, 1])


In [64]:
# scientific notation
# 9.4111e-01 means 9.4111 * 10 ** (-1), 0.94111

# turn off scientific notation
torch.set_printoptions(sci_mode=False)

In [66]:
out_1.shape

torch.Size([1000, 1])

In [68]:
# flattening, or squeezing the tensor
out_1[:, 0].shape # if you select only one index in a certain dimension,
# it will delete that dimension 


torch.Size([1000])

In [ ]:
out_1.squeeze() # this deletes every dimension with size 1

In [70]:
# Let's suppose that out_1 is our model's prediction for a given input
# out_1[n] is the prediction value of whether the n-th input data is category 1

# our target value is data_label
data_label

tensor([0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
        0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
        0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
        0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
        0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
        0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
        0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.,
        1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
        0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
        0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
        0., 0., 1., 1., 1., 1., 0., 0., 

In [75]:
# Let's define how bad our model is.
pred = out_1.squeeze()

pred[:10], data_label[:10]
torch.abs(pred-data_label)

tensor([    5.2484,    18.3531,    27.2993,    13.5547,    24.9371,    15.4235,
           31.1061,     1.9519,     4.5414,    26.1178,     6.3132,     4.3333,
            3.3074,     3.0512,    19.4584,    18.2753,     3.8892,    13.9980,
            5.3480,    10.0630,     1.2623,     6.4788,     6.9836,    12.1208,
            7.9826,     6.1282,     8.4529,     7.1248,    21.8367,     1.3863,
            2.5178,     5.7412,     2.5511,     4.1078,     8.9101,     7.1181,
           29.6483,    16.9841,    15.0757,     2.3522,     0.1663,     0.7554,
            4.3613,     0.2320,    12.2196,    19.0414,    27.8793,    14.5732,
            0.1468,     1.8627,    17.8519,     2.5384,     3.4768,    14.7084,
           20.0119,    19.3893,     2.1314,     3.5868,    25.0628,    14.4338,
            7.4742,    26.5770,     7.6751,     5.6576,    21.8059,     3.8442,
            9.5953,    12.1559,     4.1130,     7.4440,    14.5478,     4.9913,
            3.8095,     6.8092,     1.54